# Monochromatic Neural Étendue Expansion Experimental Code

### This notebook can be used to produce the monochromatic étendue expanded experimental holograms shown in the manuscript and in the supplementary information.

### In the cells below please select one expander type and one target image. For example, to produce a 64x étendue expanded hologram with the neural étendue expander please select 'neural_mono_64x'. To produce a 16x étendue expanded hologram with a random expander [Kuo et al. 2020] please select 'random_16x'. To produce a conventional hologram [Shi et al. 2021] please select 'conventional_16x' or 'conventional_64x'. The target images provided are labeled as '000.png', '001.png', and so on.

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
from imageio import imread

In [ ]:
### --- BEGIN CONFIG --- ###
# Choose only one of the following expanders.
#expander_type = 'conventional_16x'
#expander_type = 'conventional_64x'
#expander_type = 'random_16x'
#expander_type = 'random_64x'
#expander_type = 'neural_mono_16x'
expander_type = 'neural_mono_64x'

# Choose only one of the following target images.
#target_img_name = '000'
target_img_name = '001'
### ---   END CONFIG --- ###

if (expander_type == 'random_16x') or (expander_type == 'neural_mono_16x'):
    correction_factor = np.array([1.0])
    top_percentile = 99.9
    eff_corners = None
elif (expander_type == 'conventional_16x'):
    correction_factor = np.array([1.0])
    top_percentile = 99.9
    eff_corners = [144,144,240,240]
elif (expander_type == 'random_64x') or (expander_type == 'neural_mono_64x'):
    correction_factor = np.array([1.0])
    top_percentile = 99.0
    eff_corners = None
elif (expander_type == 'conventional_64x'):
    correction_factor = np.array([1.0])
    top_percentile = 99.9
    eff_corners = [336,336,432,432]
else:
    assert('Undefined expander.')
print(eff_corners)

In [ ]:
def get_target_img(img_name):
    img = imread(img_name)
    if img.dtype == 'uint8':
        img = img.astype(np.float32) / 255.0
    elif img.dtype == 'uint16':
        img = img.astype(np.float32) / 65535.0
    elif img.dtype == 'float32':
        img = img / 1.0
    else:
        assert('Invalid image type')
    if len(img.shape) == 3:
        img = 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2] # Convert to gray scale
    return img

def white_balance(cap_img, target_img, correction_factor, top_percentile, eff_corners = None):
    # Normalize so that max == 1.
    # We are only adjusting the ratios between the colors, not the overall brightness.
    if eff_corners == None:
        scale_factors = np.mean(target_img, axis=(0,1)) / np.mean(cap_img, axis=(0,1))
    else:
        cap_img_eff = cap_img[eff_corners[0]:eff_corners[2],eff_corners[1]:eff_corners[3]]
        scale_factors = np.mean(target_img, axis=(0,1)) / np.mean(cap_img_eff, axis=(0,1))

    scale_factors = scale_factors / np.max(scale_factors)
    cap_img = cap_img * scale_factors

    # Additional manual color balancing.
    cap_img = cap_img * correction_factor

    # Scale overall brightness so that top percentile of pixels are clipped.
    top_scale = np.percentile(cap_img, top_percentile)
    cap_img = cap_img / top_scale
    return np.clip(cap_img, 0.0, 1.0)

### Display Intensity Scaled Experimentally Captured Hologram ###

In [ ]:
target_img = get_target_img(os.path.join('Target_Images',target_img_name+'.png'))
w_cap = np.load(os.path.join('Data',expander_type,'w',target_img_name+'.npy'))
w_cap_wb = np.fliplr(white_balance(w_cap, target_img, correction_factor, top_percentile, eff_corners))
w_cap_wb = np.stack([w_cap_wb, w_cap_wb, w_cap_wb], axis=-1)

plt.figure()
plt.title('Experimentally Captured Hologram')
plt.imshow(w_cap_wb)